# 2.1. 用llamaindex开发新人答疑机器人

## 🚄 前言 

LlamaIndex是一个易于使用的工具，可以帮助你快速构建RAG（Retrieval-Augmented Generation）应用。它只需简单的几步，就能让你高效查询和管理数据。通过本教程的学习，你可以轻松加载数据并构建索引，快速得到所需信息。此外，LlamaIndex兼容DashScope API，能够生成高质量的自然语言响应，非常适合初学者和开发者。

## 🍁 课程目标

学完本课程后，你将能够：<br>
- 学会通过LlamaIndex框架调用通义千问模型的方法
- 掌握通过LlamaIndex框架快速创建RAG应用的过程
- 了解RAG应用创建过程中的进阶用法


## 📖 课程目录

- [1. 计算环境准备](#💻-1-计算环境准备)
- [2. 向通义千问提问](#💬-2-向通义千问提问)
- [3. 实现RAG问答](#📚-3-实现rag问答)
- [4. 开发RAG应用](#⚙️-4-开发rag应用)
    - [4.1. 保存与加载索引文件](#41-保存与加载索引文件)
    - [4.2. 修改prompt模板](#42-修改prompt模板)
    - [4.3. 选择召回文本段个数](#43-选择召回文本段个数)
    - [4.4. 理解ReRank和相似度阈值](#44-理解rerank和相似度阈值)
    - [4.5. 句子窗口检索](#45-句子窗口检索)
    - [4.6. 自动合并检索](#46-自动合并检索)   
- [拓展阅读](#拓展阅读)


## 💻 1. 计算环境准备
### 1.1. 安装依赖


In [ ]:
! pip install -r requirements.txt -q

### 1.2. 导入依赖

In [ ]:
import os
import getpass
from llama_index.core import (
    VectorStoreIndex,
    Settings,
    SimpleDirectoryReader,
    PromptTemplate,
    get_response_synthesizer,
    StorageContext,
    load_index_from_storage
)
from llama_index.embeddings.dashscope import (
    DashScopeEmbedding,
    DashScopeTextEmbeddingModels,
    DashScopeTextEmbeddingType
)
from llama_index.core.node_parser import SentenceWindowNodeParser,HierarchicalNodeParser
from llama_index.core.schema import Document
from llama_index.llms.dashscope import DashScope
from llama_index.core.schema import TextNode,NodeRelationship, RelatedNodeInfo,Document
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor,MetadataReplacementPostProcessor
from llama_index.core.base.llms.types import MessageRole, ChatMessage
from llama_index.postprocessor.dashscope_rerank import DashScopeRerank
from llama_index.core.response.notebook_utils import display_response,display_source_node
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.node_parser import get_leaf_nodes,get_root_nodes
from llama_index.core.storage.docstore import SimpleDocumentStore

### 1.3. 设置环境变量
运行下方程序，按照指引进行操作即可在notebook里引入环境变量。

In [ ]:
import sys
sys.path.append("../")
from config.load_key import load_key
load_key()
print(os.environ["DASHSCOPE_API_KEY"])

## 💬 2. 向通义千问提问
此处向你介绍通过LlamaIndex向通义千问模型提问的方法。

In [ ]:
# 定义 LLM_MODEL
LLM_MODEL = DashScope(model_name="qwen-plus")
# 定义输入到 LLM_MODEL中的messages，你可以在此定义system message与user message。system message用于指定大模型的人设、回复方式等，user message用于接收用户的输入内容。
messages = [ChatMessage(role=MessageRole.SYSTEM, content="你负责教育内容开发公司的答疑，你的名字叫公司小蜜，你要回答学员的问题。"),
            ChatMessage(role=MessageRole.USER, content="你好")]

### 2.1. 非流式输出

你可以使用chat方法来进行非流式输出。

In [29]:
# 在Jupyter中，不用print就可以展示单一变量
LLM_MODEL.chat(messages).message.content

'你好！我是公司小蜜，很高兴为您服务。您有什么问题或需要帮助的吗？'

### 2.2. 流式输出

你可以使用stream_chat方法进行流式输出，大模型将会一段段地返回结果。

In [30]:
responses = LLM_MODEL.stream_chat(messages)
for response in responses:
    print(response.delta, end="")

你好！我是公司小蜜，很高兴为您服务。您有什么问题或需要帮助的吗？

## 📚 3. 实现RAG问答

LlamaIndex 对于RAG问答的创建与使用十分友好。你可以参考以下内容了解通过LlamaIndex进行RAG问答的基本步骤与更多使用方法。
### 3.1. 设置RAG的默认大模型与embedding模型
LlamaIndex有许多方法都需要指定大模型与embedding模型，但是一个项目里往往只需要用到一个大模型与一个embedding模型，因此你可以通过Settings设置默认值，这样就无需在llamaindex的后续操作中再指定大模型与embedding模型了。

In [ ]:
def set_default_model():
    EMBED_MODEL = DashScopeEmbedding(
        # 你也可以使用其它通义千问embedding模型：https://help.aliyun.com/zh/model-studio/getting-started/models#3383780daf8hw
        model_name=DashScopeTextEmbeddingModels.TEXT_EMBEDDING_V2,
        text_type=DashScopeTextEmbeddingType.TEXT_TYPE_DOCUMENT,
    )
    # 你也可以使用其它通义千问大语言模型：https://help.aliyun.com/zh/model-studio/getting-started/models#9f8890ce29g5u
    LLM_MODEL = DashScope(model_name="qwen-plus")
    Settings.embed_model = EMBED_MODEL
    Settings.llm = LLM_MODEL

set_default_model()

### 3.2. 将文件夹中的文件解析为document对象
LlamaIndex提供了SimpleDirectoryReader方法，可以直接将指定文件夹中的文件加载为document对象，而后续的索引创建需要基于document对象。

为了便于后续处理，我们将所有document的文本集成到一个总的document对象中。

In [ ]:
def get_documents(path):
    documents = SimpleDirectoryReader(path).load_data()
    document = Document(text="\n\n".join([doc.text for doc in documents]))
    # 将中文的标点符号预先转化为英文标点符号，使得切分时不会使得chunk长度过长
    replacements = {
        '。': '. ',
        '！': '! ',
        '？': '? '
    }
    for src, dst in replacements.items():
        document.text = document.text.replace(src, dst)
    return document

# 获取docs文件夹中的document
document = get_documents(path="docs")

# Ignoring wrong pointing object是警告信息，此处可以忽略。

### 3.3. 一般的知识库问答
如果你需要通过LlamaIndex创建RAG应用，那么大致流程为：
1. 将解析好的文本转化为document对象；
2. 将document进行文本切分，返回node对象，每一个node对象都存放了切分好的文本段；
3. 将node加载为index（索引），index可以用于创建提问引擎（这一步需要传入embedding model，但是我们在之前通过set_default_model方法指定了默认的embedding model，因此此处无需再次指定）；
4. 使用index.as_query_engine方法得到提问引擎，提问引擎可以接收用户的输入，并生成回复。


此处展示了从创建索引-创建提问引擎-生成回复的流程。
> from_documents方法涵盖了第2步和第3步。

In [31]:
print("正在创建索引...")
# 这一步也会进行文本段的切分过程，此处使用默认切分方法
index = VectorStoreIndex.from_documents([document])
print("正在创建提问引擎...")
query_engine = index.as_query_engine(streaming=True)
print("正在生成回复...")
streaming_response = query_engine.query('需求分析使用的工具是什么？')
print("回答是：\n")
streaming_response.print_response_stream()

正在创建索引...
正在创建提问引擎...
正在生成回复...
回答是：

需求分析使用的工具有项目管理软件（如 Jira、Trello）、文档编辑器（如 Google Docs、Notion）和协作工具（如 Slack、Microsoft Teams）。这些工具可以帮助团队更好地组织和管理需求分析过程，确保需求明确且符合 SMART 原则。

## ⚙️ 4. 开发RAG应用

### 4.1. 保存与加载索引文件
1. 将索引保存到本地

你可以把当前创建的索引保存到本地，这样之后使用索引只要直接加载就可以了。

In [ ]:
# 将索引保存为本地文件
index.storage_context.persist("knowledge_base/test")

2. 加载本地索引

加载索引的方法也很容易，请参考以下代码。

In [ ]:
storage_context = StorageContext.from_defaults(persist_dir="knowledge_base/test")
index = load_index_from_storage(storage_context)

3. 查看召回文本段及其相似度分数
   
在向大模型发出提问后，返回的response对象中会包含source_nodes属性，你可以从中提取召回的文本段与对应的相似度分数。

In [32]:
def show_chunk_score(response):
    for i in range(len(response.source_nodes)):
        source_node = response.source_nodes[i]
        display_source_node(source_node=source_node,source_length=300)
        print("="*100)

show_chunk_score(streaming_response)
# text为召回的文本段，Similarity为文本段与用户输入的相似度

**Node ID:** 7b7df476-37bc-4173-8164-2e8609d46a3e<br>**Similarity:** 0.4547080847655753<br>**Text:** 我会通过组织线上或线下的培训工作坊，让教师熟悉内容的使用方法，以及如何结合他们的教学实践来最大化学习效果. 

7. 数据分析与评估

最后，我会定期进行数据分析，以评估我们的教育内容的有效性与影响力. 通过分析学习者的学习数据、反馈和成绩，我可以直接了解内容的实际效果，并为未来的改进提供有力依据. 这种基于数据的决策过程将确保我们的教育内容始终与学习者的需求保持一致. 



工作流程指导

需求分析

使用工具：

项目管理软件（Jira、Trello）

文档编辑器（Google Docs、Notion）

协作工具（Slack、Microsoft Teams）

注意事项：

确...<br>

**Node ID:** ba05ac14-e060-463c-a880-5a2a59d0b94a<br>**Similarity:** 0.3978345116079808<br>**Text:** 通过协同⼯作， 我能确保内容的技术实施过程顺利进⾏， 并有效传达给⽬标受众.  与团队成员之间的沟通协调，将帮助我们共同创造出兼具教育价值和市场竞争⼒的产品.  5. 教育技术平台的应⽤ 在⼯作中， 我还需要与各种教育技术平台进⾏对接， 确保开发的内容可以在不同平台上⾃如应⽤. 我会熟悉各类学习管理系统（LMS）、 移 动 学 习 应 ⽤ 以 及 其 他 在 线 教 育 ⼯ 具 的 功 能 ，以便为内容的发布和推⼴提供⽀持.  这意味着我需要不断学习这些平台的最新功能和⽤户体验设计.  6. 教师培训与⽀持 为了确保最终⽤户能够有效使⽤我们提供的教育内容， 我将参与制定教师培训计划， 设...<br>

### 4.2. 修改prompt模板

首先我们来看看LlamaIndex的默认提示词模板是什么。

In [33]:
# 查看原始模板
print(query_engine.get_prompts()['response_synthesizer:text_qa_template'].default_template.template)

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


其中，context_str为召回的文本段，query_str为用户的输入问题。

在之前的教程中，我们使用的是LlamaIndex的默认英文提示词模板，这导致模型有时可能会输出英文。在中文问答场景中，使用中文的提示词模板效果会更好。

你可以通过以下方法，将prompt更换为中文，且可以添加人设等信息，使得回答更个性化，且更符合中文问答场景。

In [ ]:
# 修改prompt模板
def update_prompt_template(query_engine):
    # 修改prompt模板
    qa_prompt_tmpl_str = (
        "你负责教育内容开发公司的答疑，你的名字叫公司小蜜，你要回答学员的问题。以下是参考信息"
        "---------------------\n"
        "{context_str}\n"
        "---------------------\n"
        "你一定要参考提供的参考信息，而不是你之前就有的知识。"
        "以友好、和善的语气回答学员的提问。在结束回答后要感谢学员的提问。"
        "学员的提问是: {query_str}\n"
        "你的回答是: "
    )
    qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)
    query_engine.update_prompts(
        {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
    )
    return query_engine

我们可以在代码中先创建一个query_engine，再用上边定义的update_prompt_template函数将其提示词更新为你自定义的模板。

In [34]:
query_engine = index.as_query_engine(streaming=True)
query_engine = update_prompt_template(query_engine)
#查看当前的prompt
print("当前prompt模板为：")
print(query_engine.get_prompts()['response_synthesizer:text_qa_template'].template)
response = query_engine.query("需求分析使用的工具是什么？")
response.print_response_stream()
print("\n")
# 打印召回文本段与相似度分数
# show_chunk_score(response=response)

当前prompt模板为：
你负责教育内容开发公司的答疑，你的名字叫公司小蜜，你要回答学员的问题。以下是参考信息---------------------
{context_str}
---------------------
你一定要参考提供的参考信息，而不是你之前就有的知识。以友好、和善的语气回答学员的提问。在结束回答后要感谢学员的提问。学员的提问是: {query_str}
你的回答是: 
你好！在进行需求分析时，我们会使用以下几种工具来确保过程高效且有序：

1. **项目管理软件**：比如 Jira 或 Trello，这些工具可以帮助我们跟踪项目的进度，管理任务列表，确保每个环节都能按时完成。
2. **文档编辑器**：例如 Google Docs 或 Notion，这些工具非常适合团队协作编辑文档，可以实时查看和编辑，非常方便。
3. **协作工具**：如 Slack 或 Microsoft Teams，这些工具不仅支持即时消息交流，还能集成多种其他应用，提高团队沟通效率。

这些工具的组合使用，能够帮助我们更好地理解需求，确保项目从一开始就走在正确的道路上。希望这个回答对你有帮助！如果你还有其他问题，欢迎随时提问。谢谢你的提问！😊



你可以把上边代码的最后一行取消注释，再次运行就能看到哪些文本段为模型的回复提供了参考信息。

### 4.3. 选择召回文本段个数
召回文本段的个数决定了大模型可以参考信息的多少。你可以尝试自定义召回的文本段个数，来找到最适合的值。

In [35]:
query_engine = index.as_query_engine(
    # 设置召回文本段个数
    similarity_top_k=3,
    streaming=True
)
query_engine = update_prompt_template(query_engine)  
response = query_engine.query("需求分析使用的工具是什么？")
response.print_response_stream()
print("\n")
print(f"召回文本段个数：{len(response.source_nodes)}")
# 打印召回文本段与相似度分数
# show_chunk_score(response)

你好！在需求分析阶段，我们会使用以下工具来确保需求的明确性和可实现性：

- **项目管理软件**：如 Jira 或 Trello，用于跟踪项目的进度和任务分配。
- **文档编辑器**：如 Google Docs 或 Notion，用于编写和共享需求文档。
- **协作工具**：如 Slack 或 Microsoft Teams，用于团队成员之间的沟通和协作。

这些工具可以帮助我们更好地组织和管理需求分析的过程，确保每个环节都清晰明了。希望这个回答对你有帮助！如果你还有其他问题，欢迎随时提问。谢谢你的提问！😊

召回文本段个数：3


### 4.4. 理解ReRank和相似度阈值
向量数据库检索出来的文本段并不总是最相关的，相关的文本段可能被排到了靠后的位置，并且召回的文本段可能与你提问的问题并不相关。<br>

你可以通过rerank（重排）模型与设置相似度阈值来解决这一问题。
- rerank<br>
  一般的步骤为：先检索较大数量的文本段，再通过rerank模型进行重排序，获得更精准的排名。
- 相似度阈值<br>
  通过对已召回文本段设置相似度阈值，过滤掉相似度低于阈值的文本段。

In [36]:
query_engine = index.as_query_engine(
    # 先设置一个较大的值
    similarity_top_k=8,
    streaming=True,
    node_postprocessors=[
        # 在rerank模型中选择你想召回的文本段个数，重排模型选择通义实验室的gte-rerank模型
        DashScopeRerank(top_n=3, model="gte-rerank"),
        # 设置一个相似度阈值，低于该阈值的文本段会被过滤掉
        SimilarityPostprocessor(similarity_cutoff=0.2)])
# 更新提示词模板
query_engine = update_prompt_template(query_engine)
response = query_engine.query("需求分析使用的工具是什么？")
response.print_response_stream()
# 打印召回文本段与相似度分数
# show_chunk_score(response)

在进行需求分析时，我们会使用以下工具来确保过程的高效和准确：

- **项目管理软件**：如Jira、Trello，这些工具可以帮助我们跟踪项目的进度，管理任务列表，确保每个阶段的任务都能按时完成。
- **文档编辑器**：比如Google Docs、Notion，这些工具非常适合团队协作编辑文档，可以实时查看和编辑，方便团队成员之间的沟通和协作。
- **协作工具**：例如Slack、Microsoft Teams，这些工具不仅支持即时消息交流，还可以集成其他应用，提高团队的工作效率。

这些工具的使用有助于确保需求分析阶段的信息透明度，促进团队成员之间的有效沟通，同时也便于收集和整理来自各方的反馈和建议。希望这个回答对您有所帮助！如果您还有其他问题，欢迎随时提问。谢谢您的提问！

### 4.5. 句子窗口检索
句子窗口检索是一种能够在检索阶段排除噪声干扰信息，在生成阶段补充信息的方法。具体步骤如下：
1. 将文本按照"."、"?"、"!"切分成chunk；
2. 设置window_size参数，构建K-V对。K值为第一步切分的chunk；V值为以该chunk为中心，向左右两侧各寻找window_size个chunk。
3. 在检索时，用query与第一步切分的chunk对比，将检索出的chunk对应的V值取出，通过后处理的模块将其作为生成时的参考内容。


总结起来就是：**句子窗口检索**通过先检索出最相关的句子，然后返回围绕该句子的更广泛的文本，为生成阶段提供更丰富的上下文，从而提高生成内容的相关性和准确性。

In [ ]:
node_parser = SentenceWindowNodeParser.from_defaults(
    # 在这里设置窗口大小
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)
nodes = node_parser.get_nodes_from_documents([document])

1. 在进行句子滑窗检索前（没有开启后处理）

In [ ]:
index = VectorStoreIndex(nodes)
query_engine = index.as_query_engine(similarity_top_k=1)
response = query_engine.query("开发阶段的注意事项是什么？")
display_response(response)

In [ ]:
show_chunk_score(response)

2. 进行句子滑窗后（开启后处理模块）

In [ ]:
window_text_replace = MetadataReplacementPostProcessor(
    target_metadata_key="window"
)
query_engine = index.as_query_engine(
    similarity_top_k=1,
    node_postprocessors=[window_text_replace])
response = query_engine.query("开发阶段的注意事项是什么？")
display_response(response)

In [ ]:
show_chunk_score(response)

### 4.6. 自动合并检索
与句子窗口检索类似，自动合并检索也是先检索出最相关的句子，然后返回围绕该句子的更广泛的文本。它在切割文本段的时候会按照定义的层级结构进行切分。

假设chunk_sizes=[512, 128]，那么在切割文本段时，叶节点每一个chunk的最长字符串长度为128；父节点会包含它的叶节点，每一个chunk的最长字符串长度为512。

在检索时，自动合并检索方法会按照叶节点进行检索，然后对父节点进行统计：如果某一父节点的多数叶节点（比例可以设置，通过simple_ratio_thresh参数设置）被召回，那么就把该父节点作为上下文传入大模型；如果每一个父节点都没有多数叶节点被召回，那么就把单独的叶节点作为上下文传入大模型。

In [ ]:
# 使用默认设置创建层次结构节点解析器
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[512, 128]
)
nodes = node_parser.get_nodes_from_documents([document])

In [ ]:
# 获取叶节点并打印其中一个的文本作为示例
leaf_nodes = get_leaf_nodes(nodes)
print(leaf_nodes[1].text)

In [ ]:
docstore = SimpleDocumentStore()
docstore.add_documents(nodes)
storage_context = StorageContext.from_defaults(docstore=docstore)

In [ ]:
index = VectorStoreIndex(leaf_nodes)
base_retriever = index.as_retriever(similarity_top_k=7)
retriever = AutoMergingRetriever(base_retriever, storage_context, verbose=True)
query_engine = RetrieverQueryEngine.from_args(retriever)
response = query_engine.query("开发阶段的注意事项是什么？")
display_response(response)

In [ ]:
show_chunk_score(response)

## 拓展阅读

如果你想要创建以输入框形式交互的RAG应用，请阅读：[百炼RAG实践教程文档](https://help.aliyun.com/zh/model-studio/use-cases/build-rag-application-based-on-local-retrieval)。

<img src="https://help-static-aliyun-doc.aliyuncs.com/assets/img/zh-CN/4139126271/p844266.gif" alt="我的notebook" width="600px">

你可以在下载[文档中的压缩包](https://help.aliyun.com/zh/model-studio/use-cases/build-rag-application-based-on-local-retrieval#b91232e354cb4)后，找到chat.py中的get_model_response函数并进行修改，适配到本教程介绍的方法。

## ✅ 本节小结
通过学习本节课程，你已经掌握了LlamaIndex的基本用法。在下一节中，你将学习到如何通过优化提示词来提升答疑机器人的回复效果。

## 🔥 课后小测验


【单选题】 2.1.1. 以下代码片段的作用是什么？（ ）
```python
index.storage_context.persist(persist_dir=PERSIST_DIR)
```
A. 从磁盘加载嵌入向量。

B. 将嵌入向量存储到内存中。

C. 将嵌入向量持久化到指定的目录。

D. 创建一个新的 StorageContext 对象。

答案：C